# Launch a VM

In [ ]:
from chi import server, context
import chi, os, time, datetime
import swiftclient

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [ ]:
s = server.Server(
    f"node1-project16",
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.medium"
)

s.submit(idempotent=True)

In [ ]:
s.associate_floating_ip()
s.refresh()
s.show(type="widget")

In [ ]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:
  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')
  nova_server.add_security_group(sg['name'])

s.refresh()
s.check_connectivity()

## Bring up a block storage and mount

In [ ]:
cinder_client = chi.clients.cinder()

volume = cinder_client.volumes.create(name=f"block-persist-project16", size=4)

server_id = chi.server.get_server(f"node1-project16").id
volume_manager = chi.nova().volumes
volume_manager.create_server_volume(server_id = s.id, volume_id = volume.id, device="/dev/vdb")

In [ ]:
s.execute("lsblk")

In [ ]:
s.execute("sudo parted -s /dev/vdb mklabel gpt")
s.execute("sudo parted -s /dev/vdb mkpart primary ext4 0% 100%")
s.execute("sudo mkfs.ext4 /dev/vdb1")
s.execute("sudo mkdir -p /mnt/block")
s.execute("sudo mount /dev/vdb1 /mnt/block")
s.execute("sudo chown -R cc /mnt/block")
s.execute("sudo chgrp -R cc /mnt/block")
s.execute("df -h")

## Bring up an Object Storage

In [ ]:
context.choose_project()
context.choose_site(default="CHI@TACC")

os_conn = chi.clients.connection()
token = os_conn.authorize()
storage_url = os_conn.object_store.get_endpoint()

swift_conn = swiftclient.Connection(preauthurl=storage_url,
                                    preauthtoken=token,
                                    retries=5)

container_name = "object-persist-project16"

swift_conn.put_container(container_name)